# Rundown of analysis, training, and prediction of multichannel segmentation

The whole process consists of several steps that need to be repeated when training networks for different contrasts or resolutions.
Here, we try to train a dedicated network for the AD/PCA highres data.
The input for the segmentation will be

- 500 um resolution scans of the whole head
- we train on multichannel data that consists of PD, R1, and R2s maps
- the plain input maps will be clipped and rescaled to suitable value ranges (because noisy regions contain large outliers that would reduce the range of meaningful contrast for the network to learn from)

To run the whole pipeline, the following steps were necessary:

1. Decide how many different contrasts you want as input for your network. Remember that for this approach the contrasts need to be perfectly co-registered. At the moment, I believe this makes the most sense when you're using multiple echoes or, like here, quantitative maps that show different features.
2. Inspect the images and find suitable value ranges for all contrasts. These ranges are then used to clip and rescale the maps for further processing.
3. For at least one set of input images, you need a segmentation. It doesn't need to be a perfect segmentation, but it will be used to calculate the regional gray value statistics for each segmentation class.
4. From this analysis, a dedicated _generator configuration_ is created for you. It contains the specification for the random distribution for every segmentation class. This specification is used to create random synthetic maps that should resemble your original data as close as possible. In the case here, this will not be perfect because maps like R2s have gray value distributions in some regions that can not really be modeled with a Gaussian.
5. The generator configuration is used to create the training data for the neural network. There are two options: You can create the training data on-the-fly, or you can create them beforehand and store them in a TensorFlow format that is easy to deserialize. The latter approach has another advantage. You can train different network parametrizations with the same training data and compare the performance.
6. With the training data (or the generation configuration), you derive a training configuration that further specifies the used network model and its hyperparameters.
7. During the training, we write out the weights and biases of the model as snapshots. These can be loaded back if you want to create a network for prediction (aka running the actual segmentation)

## 1. & 2. Used Contrasts and Value Ranges

## 2. & 3. Running the Region Analysis and Create the Generator Config

## 4. & 5. Creating the Training Data

## 6. Training Configuration and Training on HPC

## 7. Prediction
 